In [121]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import sys
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls 

In [109]:
tumors = pd.read_csv('input/planilhadetrabalhoestrodrigoegustavo2.csv', encoding='latin1')
del tumors['casenumber']
tumors.head(10)

,sample,Mass,atdiagnosis,stage,histopathology,disease,cytometry,Neventos,viabilidade,celtumoral,...,EPCAMtotal,CD9MFI,CD10MFI,CD34MFI,CD38MFI,CD57MFI,CD58MFI,CD81MFI,CD90MFI,CD117MFI
0,abdominal,yes,yes,metastatic,neuroblastoma,neuroblastoma,neuroblastoma,1777355.0,7.0,76.8,...,875.00,7132.00,136.00,2503.00,209.0,625.00,622.0,4489.00,5433.00,229.00
1,abdominal,yes,no,,neuroblastoma,neuroblastoma,neuroblastoma,153916.0,6.0,10.0,...,105.00,4982.34,-920.00,1852.00,-16.0,50.90,133.4,1000.98,5834.92,158.88
2,BM,no,,metastatic,disease free,neuroblastoma,disease free,150000.0,96.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abdominal,yes,yes,local,ganglioneuroblastoma,ganglioneuroblastoma,neuroblastoma,450000.0,13.0,93.0,...,1024.00,23361.00,-83.00,NaN,-77.0,981.00,NaN,10429.00,2680.00,262.00
4,abdominal,yes,yes,,neuroblastoma,neuroblastoma,neuroblastoma,408081.0,51.5,6.0,...,-163.00,2081.00,-47.00,1944.00,-2850.0,4997.96,199.0,3567.18,4130.00,1290.00
5,lymph node,yes,,metastatic,neuroblastoma,neuroblastoma,neuroblastoma,382319.0,92.6,10.3,...,-68.00,909.00,-12.00,206.00,519.0,969.00,98.0,3727.00,3182.00,226.00
6,PB,no,yes,metastatic,,neuroblastoma,disease free,300000.0,95.2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,abdominal,yes,yes,local,adrenal carcinoma,adrenal carcinoma,non-hematopoietic solid tumor,400000.0,70.3,82.8,...,NaN,1953.90,127.95,632.67,-104.3,126.75,1142.6,-6.03,2677.59,100.53
8,head and neck,yes,yes,,neuroblastoma,neuroblastoma,neuroblastoma,152397.0,43.5,27.2,...,126.00,2752.00,147.00,1097.00,-9.6,579.00,830.0,10749.00,2303.00,942.00
9,BM,no,,metastatic,neuroblastoma,neuroblastoma,neuroblastoma,800000.0,79.0,41.3,...,1.88,3223.98,34.00,743.00,-1905.0,117.50,276.0,5848.00,4044.00,70.80


#### Listamos quais são os grupos de tumores que temos dados

In [110]:
all_groups = tumors.disease.unique()
for key in all_groups:
    print key

neuroblastoma
ganglioneuroblastoma
adrenal carcinoma
Extraesqueletal Ewing Sarcoma
reactive
rhabdomyosarcoma
undiferenciated malignant neoplasm
wilms tumor
nasopharyngeal carcinoma
hemangioperycitoma
germ cell
neoplasia fibrohistiocitaria cutanea do tip
Anaplastic lymphoma
Diffuse Large B cell Lymphoma
B lymphoblastic lymphoma/leukemia
Burkitt Lymphoma
Hodgkin Lymphoma
EBV related lymphoma
T lymphoblastic lymphoma
histiocytic sarcoma
hepatoblastoma
chondrosarcoma
hemangioma
pheochromocytoma
mesoblastic nephroma
histiocytosis
ganglioneuroma
clear cell sarcoma
osteosarcoma
cystic nephroma
renal carcinoma
pseudopapilary pancreatic solid tumor
thyroid-like follicular carcinoma of kidney
malignant nerve sheath tumor
T-cell/histiocyte rich B cell lymphoma
neurofibromatose


In [133]:
def plot_results(disease1, disease2, labels, res_X):
    
    disease1_x = []
    disease1_y = []
    disease2_x = []
    disease2_y = []
    for index in range(0, len(labels)):
        if labels[index] == disease1:
            disease1_x.append(res_X[index,0])
            disease1_y.append(res_X[index,1])
        else:
            disease2_x.append(res_X[index,0])
            disease2_y.append(res_X[index,1])

    trace0 = go.Scatter(
        x = disease1_x,
        y = disease1_y,
        name = disease1,
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(255, 0, 0, .9)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )

    trace1 = go.Scatter(
        x = disease2_x,
        y = disease2_y,
        name = disease2,
        mode = 'markers',
        marker = dict(
            size = 10,
            color = 'rgba(0, 0, 255, .9)',
            line = dict(
                width = 2,
            )
        )
    )

    data = [trace0, trace1]
    return data
    #     layout = dict(title = disease1 + ' x ' + disease2,
    #                   yaxis = dict(zeroline = False),
    #                   xaxis = dict(zeroline = False)
    #                  )

    #title = disease1 + ' x ' + disease2
    #return dict(data=data, layout=layout), title
    
    #py.iplot(fig, filename=disease1 + ' x ' + disease2)

In [112]:
def tnse_res(raw_df):
    X = raw_df.values
    last_divergence = sys.maxint
    X_embedded = None
    for i in range(0,25):
        my_tse = TSNE(n_components=2)
        cur_X_embedded = my_tse.fit_transform(X)
        if last_divergence >= my_tse.kl_divergence_:
            X_embedded = cur_X_embedded
            last_divergence = my_tse.kl_divergence_
            
    print 'Lowest kl_divergence: ' + str(last_divergence)
    return X_embedded

In [113]:
def loop_one_on_one(disease1, disease2):
    result = []
    selected_df = tumors[(tumors.disease == disease1) ^ (tumors.disease == disease2)]
    marks_only_df = selected_df.iloc[:,11:]
    clean_df = marks_only_df.dropna(axis=1, how='any')
    if not clean_df.empty:
        print "\n"
        print disease1 + ' x ' + disease2
        display(clean_df)
        res_X = tnse_res(clean_df)
        labels = selected_df.dropna(axis=1, how='any').disease.values
        result = [disease1, disease2, labels, res_X]
        
    return result
    #plot_results(disease1, disease2, labels, res_X)
    

In [114]:
final_results = []
for j in range(0, len(all_groups) - 1):
    current_disease = all_groups[j]
    for k in range(j + 1, len(all_groups)):
        pair_disease = all_groups[k]
        res = loop_one_on_one(current_disease, pair_disease)
        if len(res) > 0:
            final_results.append(res)



ganglioneuroblastoma x adrenal carcinoma


,CD45MFI,CD56MFIPECy7,CD9MFI,CD38MFI,CD81MFI
3,-82.0,17072.0,23361.0,-77.0,10429.00
7,98.0,3803.0,1953.9,-104.3,-6.03
162,501.0,77137.0,27398.0,12.0,15080.00
163,289.0,13409.0,55119.0,87.0,13471.00
165,966.0,26626.0,66416.0,187.0,23830.00


Lowest kl_divergence: 0.110347382724


ganglioneuroblastoma x undiferenciated malignant neoplasm


,CD45MFI,CD56MFIPECy7,CD9MFI,CD38MFI,CD81MFI
3,-82.0,17072.0,23361.0,-77.0,10429.0
27,404.0,44098.0,7789.0,721.0,4860.0
162,501.0,77137.0,27398.0,12.0,15080.0
163,289.0,13409.0,55119.0,87.0,13471.0
165,966.0,26626.0,66416.0,187.0,23830.0
258,1004.0,8498.0,574.0,105.0,15182.0


Lowest kl_divergence: 0.13018335402


ganglioneuroblastoma x hemangioperycitoma


...

Lowest kl_divergence: 0.0


renal carcinoma x neurofibromatose


,CD45MFI,CD56MFIPECy7,GD2total
251,1174.0,9846.0,12260.0
308,767.0,18954.0,4758.0


Lowest kl_divergence: 0.0


pseudopapilary pancreatic solid tumor x neurofibromatose


,CD45MFI,CD56MFIPECy7,GD2total,CD9MFI,CD10MFI,CD38MFI,CD57MFI,CD81MFI,CD90MFI
259,1124.0,42899.0,1527.0,36998.0,3192.0,194.0,440.0,13065.0,133.0
308,767.0,18954.0,4758.0,2174.0,4610.0,437.0,240.0,14338.0,522.0


Lowest kl_divergence: 0.0


In [134]:
figures = []
titles = []
for res in final_results:
    print res
    fig, title = plot_results(res[0], res[1], res[2], res[3])
    figures.append(fig)
    titles.append(title)

[u'ganglioneuroblastoma', u'adrenal carcinoma', array([u'ganglioneuroblastoma', u'adrenal carcinoma',
       u'ganglioneuroblastoma', u'ganglioneuroblastoma',
       u'ganglioneuroblastoma'], dtype=object), array([[ 14.630407 ,  29.253386 ],
       [ 46.7624   , -16.743235 ],
       [ 71.83053  ,   5.860201 ],
       [ 16.640749 ,  -4.4431553],
       [ 47.200504 ,  27.265402 ]], dtype=float32)]
[u'ganglioneuroblastoma', u'undiferenciated malignant neoplasm', array([u'ganglioneuroblastoma', u'undiferenciated malignant neoplasm',
       u'ganglioneuroblastoma', u'ganglioneuroblastoma',
       u'ganglioneuroblastoma', u'undiferenciated malignant neoplasm'],
      dtype=object), array([[-35.765133,  28.531275],
       [-26.80155 ,  19.765152],
       [-37.86518 ,  40.91445 ],
       [-24.648233,  34.354843],
       [-41.345383,  17.290243],
       [-48.19355 ,  30.37007 ]], dtype=float32)]
[u'ganglioneuroblastoma', u'hemangioperycitoma', array([u'ganglioneuroblastoma', u'hemangioperycitom

In [139]:
print len(titles)
figure = tls.make_subplots(rows=15, cols=3)

45
This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]   
[ (2,1) x4,y4 ]     [ (2,2) x5,y5 ]     [ (2,3) x6,y6 ]   
[ (3,1) x7,y7 ]     [ (3,2) x8,y8 ]     [ (3,3) x9,y9 ]   
[ (4,1) x10,y10 ]   [ (4,2) x11,y11 ]   [ (4,3) x12,y12 ] 
[ (5,1) x13,y13 ]   [ (5,2) x14,y14 ]   [ (5,3) x15,y15 ] 
[ (6,1) x16,y16 ]   [ (6,2) x17,y17 ]   [ (6,3) x18,y18 ] 
[ (7,1) x19,y19 ]   [ (7,2) x20,y20 ]   [ (7,3) x21,y21 ] 
[ (8,1) x22,y22 ]   [ (8,2) x23,y23 ]   [ (8,3) x24,y24 ] 
[ (9,1) x25,y25 ]   [ (9,2) x26,y26 ]   [ (9,3) x27,y27 ] 
[ (10,1) x28,y28 ]  [ (10,2) x29,y29 ]  [ (10,3) x30,y30 ]
[ (11,1) x31,y31 ]  [ (11,2) x32,y32 ]  [ (11,3) x33,y33 ]
[ (12,1) x34,y34 ]  [ (12,2) x35,y35 ]  [ (12,3) x36,y36 ]
[ (13,1) x37,y37 ]  [ (13,2) x38,y38 ]  [ (13,3) x39,y39 ]
[ (14,1) x40,y40 ]  [ (14,2) x41,y41 ]  [ (14,3) x42,y42 ]
[ (15,1) x43,y43 ]  [ (15,2) x44,y44 ]  [ (15,3) x45,y45 ]



In [140]:
for x in range(0, len(titles)):
    figure.append_trace(figures[x], x+1, 1)

Exception: The (row, col) pair sent is out of range. Use Figure.print_grid to view the subplot grid. 

In [141]:
py.iplot(figure)